In [ ]:
from PIL import Image
import numpy as np
import cv2
from scipy.fft import fft2, ifft2
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt


# Load the image
image_path = "geldbabe-cropped.jpeg"
original_image = Image.open(image_path)
gray_image = cv2.cvtColor(np.array(original_image), cv2.COLOR_BGR2GRAY)

# Create a Gaussian PSF (Point Spread Function)
psf_size = 8  # You can adjust this size
psf_sigma = 6
K = 0.01  # Regularization term, you can adjust this value


gaussian_psf = np.zeros((psf_size, psf_size))
gaussian_psf[psf_size//2, psf_size//2] = 1
gaussian_psf = gaussian_filter(gaussian_psf, sigma=psf_sigma)  # Adjust the sigma value for the blur size
gaussian_psf /= gaussian_psf.sum()  # Normalize

# Apply FFT to the Gaussian PSF
H_gaussian = fft2(gaussian_psf, gray_image.shape)

# Apply Wiener deconvolution
F_gaussian = np.conj(H_gaussian) / (np.abs(H_gaussian)**2 + K)
deconvolved_freq_gaussian = fft2(gray_image) * F_gaussian

# Inverse FFT to get the deconvolved image
deconvolved_image_gaussian = np.abs(ifft2(deconvolved_freq_gaussian))

# Save the deconvolved image
output_path = "out/deconvolved_image.png"
Image.fromarray(np.uint8(deconvolved_image_gaussian)).save(output_path)

plt.figure(figsize=(8, 8))
plt.title("Convoluted Image")
plt.axis('off')
plt.imshow(deconvolved_image_gaussian)
plt.show()